In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Notebook 1: Exploração Inicial dos Dados e Pré-processamento\n",
    "\n",
    "Este notebook realiza uma exploração inicial dos modelos 3D de crânios (arquivos `.stl`) e demonstra as etapas de pré-processamento implementadas no módulo `mesh_processor.py`.\n",
    "\n",
    "**Objetivos:**\n",
    "1. Carregar um ou mais modelos STL.\n",
    "2. Visualizar a malha original.\n",
    "3. Aplicar a simplificação da malha.\n",
    "4. Visualizar a malha simplificada.\n",
    "5. Verificar o funcionamento do cache.\n",
    "6. Analisar propriedades básicas das malhas (número de vértices/faces)."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Configurações Iniciais e Imports\n",
    "import os\n",
    "import sys\n",
    "import logging\n",
    "import time\n",
    "import trimesh\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "\n",
    "# Adicionar o diretório raiz do projeto ao path\n",
    "module_path = os.path.abspath(os.path.join('..'))\n",
    "if module_path not in sys.path:\n",
    "    sys.path.append(module_path)\n",
    "\n",
    "from src.core.mesh_processor import MeshProcessor\n",
    "from src.utils.visualization import plot_landmarks_2d, plot_landmarks, OPEN3D_AVAILABLE\n",
    "from src.utils.helpers import setup_logging\n",
    "\n",
    "# Configurar logging\n",
    "setup_logging(log_level=logging.INFO)\n",
    "\n",
    "print(f\"Open3D disponível: {OPEN3D_AVAILABLE}\")\n",
    "print(f\"Diretório de trabalho: {os.getcwd()}\")\n",
    "print(f\"Path do módulo: {module_path}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Configurar diretórios\n",
    "BASE_DIR = module_path\n",
    "DATA_DIR = os.path.join(BASE_DIR, 'data', 'skulls')\n",
    "CACHE_DIR = os.path.join(BASE_DIR, 'data', 'cache')\n",
    "RESULTS_DIR = os.path.join(BASE_DIR, 'results')\n",
    "\n",
    "# Criar diretórios se não existirem\n",
    "os.makedirs(DATA_DIR, exist_ok=True)\n",
    "os.makedirs(CACHE_DIR, exist_ok=True)\n",
    "os.makedirs(RESULTS_DIR, exist_ok=True)\n",
    "\n",
    "print(f\"Diretório de dados: {DATA_DIR}\")\n",
    "print(f\"Diretório de cache: {CACHE_DIR}\")\n",
    "print(f\"Diretório de resultados: {RESULTS_DIR}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Criar arquivo STL dummy para demonstração\n",
    "dummy_stl_filename = \"dummy_skull_explore.stl\"\n",
    "dummy_stl_path = os.path.join(DATA_DIR, dummy_stl_filename)\n",
    "\n",
    "if not os.path.exists(dummy_stl_path):\n",
    "    print(f\"Criando arquivo STL dummy em {dummy_stl_path}\")\n",
    "    # Criar uma esfera achatada para simular um crânio\n",
    "    mesh_dummy = trimesh.primitives.Sphere(radius=60, subdivisions=4)\n",
    "    mesh_dummy.apply_scale([1, 1, 0.7])  # Achatar um pouco\n",
    "    mesh_dummy.vertices += [0, 0, 60]  # Mover para cima\n",
    "    mesh_dummy.export(dummy_stl_path)\n",
    "    print(f\"Arquivo STL dummy '{dummy_stl_filename}' criado.\")\n",
    "else:\n",
    "    print(f\"Usando arquivo STL existente: '{dummy_stl_filename}'\")\n",
    "\n",
    "# Arquivo alvo para análise\n",
    "TARGET_STL_FILENAME = dummy_stl_filename\n",
    "print(f\"Arquivo alvo: {TARGET_STL_FILENAME}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Carregamento da Malha Original\n",
    "\n",
    "Utilizamos a classe `MeshProcessor` para carregar o arquivo STL. O processador lida com o cache para acelerar carregamentos futuros."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Inicializar processador\n",
    "processor = MeshProcessor(data_dir=DATA_DIR, cache_dir=CACHE_DIR)\n",
    "\n",
    "# Carregar a malha (primeira vez pode demorar mais se não houver cache)\n",
    "print(f\"Carregando malha: {TARGET_STL_FILENAME}\")\n",
    "start_time = time.time()\n",
    "mesh_original = processor.load_skull(TARGET_STL_FILENAME, use_cache=True)\n",
    "load_time = time.time() - start_time\n",
    "\n",
    "if mesh_original:\n",
    "    print(f\"✅ Malha original carregada com sucesso em {load_time:.4f}s\")\n",
    "    \n",
    "    # Obter estatísticas da malha\n",
    "    stats = processor.get_mesh_stats(mesh_original)\n",
    "    print(\"\\n=== Estatísticas da Malha Original ===\")\n",
    "    print(f\"Vértices: {stats['vertices']:,}\")\n",
    "    print(f\"Faces: {stats['faces']:,}\")\n",
    "    print(f\"Arestas: {stats['edges']:,}\")\n",
    "    print(f\"Volume: {stats['volume']:.2f} mm³\" if stats['volume'] else \"Volume: N/A\")\n",
    "    print(f\"Área de superfície: {stats['surface_area']:.2f} mm²\")\n",
    "    print(f\"Watertight: {stats['is_watertight']}\")\n",
    "    print(f\"Centroide: [{stats['centroid'][0]:.2f}, {stats['centroid'][1]:.2f}, {stats['centroid'][2]:.2f}]\")\n",
    "    print(f\"Extensões: [{stats['extents'][0]:.2f}, {stats['extents'][1]:.2f}, {stats['extents'][2]:.2f}]\")\n",
    "else:\n",
    "    print(f\"❌ Falha ao carregar a malha {TARGET_STL_FILENAME}\")\n",
    "    print(\"Verifique se o arquivo existe e está em formato válido.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Visualização da Malha Original\n",
    "\n",
    "Visualizamos a malha carregada usando projeções 2D. Se Open3D estiver disponível, também pode ser visualizada em 3D."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if mesh_original:\n",
    "    print(\"Gerando visualização da malha original...\")\n",
    "    \n",
    "    # Salvar visualização 2D\n",
    "    vis_orig_2d_path = os.path.join(RESULTS_DIR, f\"{os.path.splitext(TARGET_STL_FILENAME)[0]}_original_2d.png\")\n",
    "    \n",
    "    success = plot_landmarks(\n",
    "        mesh_original, \n",
    "        landmarks_dict=None,  # Sem landmarks nesta etapa\n",
    "        title=f\"Malha Original: {TARGET_STL_FILENAME}\", \n",
    "        use_3d=False,  # Forçar 2D para notebook\n",
    "        save_path_2d=vis_orig_2d_path\n",
    "    )\n",
    "    \n",
    "    if success and os.path.exists(vis_orig_2d_path):\n",
    "        print(f\"✅ Visualização 2D salva em: {vis_orig_2d_path}\")\n",
    "        \n",
    "        # Exibir a imagem no notebook (opcional)\n",
    "        from IPython.display import Image, display\n",
    "        try:\n",
    "            print(\"\\n=== Visualização da Malha Original ===\")\n",
    "            display(Image(filename=vis_orig_2d_path))\n",
    "        except:\n",
    "            print(\"Não foi possível exibir a imagem no notebook\")\n",
    "    else:\n",
    "        print(\"❌ Falha ao gerar visualização\")\n",
    "else:\n",
    "    print(\"❌ Malha original não carregada, impossível visualizar.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Simplificação da Malha\n",
    "\n",
    "Aplicamos o algoritmo de decimação quadrática para reduzir o número de faces da malha. Isso é crucial para otimizar o desempenho dos algoritmos de detecção em hardware limitado."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "TARGET_FACES = 2000  # Alvo de faces para simplificação\n",
    "\n",
    "if mesh_original:\n",
    "    print(f\"Simplificando a malha para aproximadamente {TARGET_FACES} faces...\")\n",
    "    \n",
    "    start_time = time.time()\n",
    "    mesh_simplified = processor.simplify(\n",
    "        mesh_original, \n",
    "        target_faces=TARGET_FACES, \n",
    "        use_cache=True, \n",
    "        original_filename=TARGET_STL_FILENAME\n",
    "    )\n",
    "    simplify_time = time.time() - start_time\n",
    "    \n",
    "    if mesh_simplified:\n",
    "        print(f\"✅ Malha simplificada com sucesso em {simplify_time:.4f}s\")\n",
    "        \n",
    "        # Obter estatísticas da malha simplificada\n",
    "        stats_simplified = processor.get_mesh_stats(mesh_simplified)\n",
    "        stats_original = processor.get_mesh_stats(mesh_original)\n",
    "        \n",
    "        print(\"\\n=== Comparação Original vs Simplificada ===\")\n",
    "        print(f\"Vértices: {stats_original['vertices']:,} → {stats_simplified['vertices']:,} \"\n",
    "              f\"({(1 - stats_simplified['vertices']/stats_original['vertices'])*100:.1f}% redução)\")\n",
    "        print(f\"Faces: {stats_original['faces']:,} → {stats_simplified['faces']:,} \"\n",
    "              f\"({(1 - stats_simplified['faces']/stats_original['faces'])*100:.1f}% redução)\")\n",
    "        \n",
    "        if stats_original['volume'] and stats_simplified['volume']:\n",
    "            volume_preservation = (stats_simplified['volume'] / stats_original['volume']) * 100\n",
    "            print(f\"Preservação de volume: {volume_preservation:.1f}%\")\n",
    "        \n",
    "        area_preservation = (stats_simplified['surface_area'] / stats_original['surface_area']) * 100\n",
    "        print(f\"Preservação de área: {area_preservation:.1f}%\")\n",
    "        \n",
    "    else:\n",
    "        print(\"❌ Falha ao simplificar a malha. Usando malha original.\")\n",
    "        mesh_simplified = mesh_original\n",
    "else:\n",
    "    print(\"❌ Malha original não carregada, impossível simplificar.\")\n",
    "    mesh_simplified = None"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Visualização da Malha Simplificada\n",
    "\n",
    "Visualizamos a malha após a simplificação para comparar com a original."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if mesh_simplified:\n",
    "    print(\"Gerando visualização da malha simplificada...\")\n",
    "    \n",
    "    vis_simpl_2d_path = os.path.join(\n",
    "        RESULTS_DIR, \n",
    "        f\"{os.path.splitext(TARGET_STL_FILENAME)[0]}_simplified_{TARGET_FACES}f_2d.png\"\n",
    "    )\n",
    "    \n",
    "    success = plot_landmarks(\n",
    "        mesh_simplified, \n",
    "        landmarks_dict=None, \n",
    "        title=f\"Malha Simplificada ({len(mesh_simplified.faces)} faces): {TARGET_STL_FILENAME}\", \n",
    "        use_3d=False,\n",
    "        save_path_2d=vis_simpl_2d_path\n",
    "    )\n",
    "    \n",
    "    if success and os.path.exists(vis_simpl_2d_path):\n",
    "        print(f\"✅ Visualização 2D da malha simplificada salva em: {vis_simpl_2d_path}\")\n",
    "        \n",
    "        # Exibir a imagem no notebook\n",
    "        from IPython.display import Image, display\n",
    "        try:\n",
    "            print(\"\\n=== Visualização da Malha Simplificada ===\")\n",
    "            display(Image(filename=vis_simpl_2d_path))\n",
    "        except:\n",
    "            print(\"Não foi possível exibir a imagem no notebook\")\n",
    "    else:\n",
    "        print(\"❌ Falha ao gerar visualização da malha simplificada\")\n",
    "else:\n",
    "    print(\"❌ Malha simplificada não disponível para visualização.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Verificação do Cache\n",
    "\n",
    "Executamos novamente o carregamento e a simplificação para verificar se o cache está sendo utilizado (o tempo de execução deve ser significativamente menor)."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if mesh_original:\n",
    "    print(\"\\n=== Testando Desempenho do Cache ===\")\n",
    "    \n",
    "    # Teste 1: Recarregamento da malha original\n",
    "    print(\"Recarregando malha original (deve usar cache)...\")\n",
    "    start_time = time.time()\n",
    "    mesh_original_cached = processor.load_skull(TARGET_STL_FILENAME, use_cache=True)\n",
    "    cached_load_time = time.time() - start_time\n",
    "    \n",
    "    if mesh_original_cached:\n",
    "        print(f\"✅ Recarregamento do cache: {cached_load_time:.4f}s\")\n",
    "        print(f\"📈 Aceleração: {load_time/cached_load_time:.1f}x mais rápido\")\n",
    "    else:\n",
    "        print(\"❌ Falha ao recarregar do cache\")\n",
    "    \n",
    "    # Teste 2: Resimplificação\n",
    "    print(\"\\nResimplificando malha (deve usar cache)...\")\n",
    "    start_time = time.time()\n",
    "    mesh_simplified_cached = processor.simplify(\n",
    "        mesh_original, \n",
    "        target_faces=TARGET_FACES, \n",
    "        use_cache=True, \n",
    "        original_filename=TARGET_STL_FILENAME\n",
    "    )\n",
    "    cached_simplify_time = time.time() - start_time\n",
    "    \n",
    "    if mesh_simplified_cached:\n",
    "        print(f\"✅ Resimplificação do cache: {cached_simplify_time:.4f}s\")\n",
    "        if 'simplify_time' in locals():\n",
    "            print(f\"📈 Aceleração: {simplify_time/cached_simplify_time:.1f}x mais rápido\")\n",
    "    else:\n",
    "        print(\"❌ Falha ao resimplificar do cache\")\n",
    "    \n",
    "    # Verificar consistência\n",
    "    if (mesh_original_cached and mesh_simplified_cached and \n",
    "        len(mesh_original_cached.vertices) == len(mesh_original.vertices) and\n",
    "        len(mesh_simplified_cached.vertices) == len(mesh_simplified.vertices)):\n",
    "        print(\"✅ Cache está funcionando corretamente - dados consistentes\")\n",
    "    else:\n",
    "        print(\"⚠️  Possível inconsistência no cache\")\n",
    "        \n",
    "else:\n",
    "    print(\"❌ Não é possível testar cache sem malha original\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Análise Adicional\n",
    "\n",
    "Realizamos análises adicionais sobre as propriedades geométricas da malha."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if mesh_simplified:\n",
    "    print(\"=== Análise Geométrica Adicional ===\")\n",
    "    \n",
    "    # Análise do comprimento das arestas\n",
    "    edge_lengths = mesh_simplified.edges_unique_length\n",
    "    \n",
    "    print(f\"\\nEstatísticas das Arestas:\")\n",
    "    print(f\"Total de arestas: {len(edge_lengths):,}\")\n",
    "    print(f\"Comprimento médio: {np.mean(edge_lengths):.4f} mm\")\n",
    "    print(f\"Comprimento mediano: {np.median(edge_lengths):.4f} mm\")\n",
    "    print(f\"Desvio padrão: {np.std(edge_lengths):.4f} mm\")\n",
    "    print(f\"Comprimento mínimo: {np.min(edge_lengths):.4f} mm\")\n",
    "    print(f\"Comprimento máximo: {np.max(edge_lengths):.4f} mm\")\n",
    "    \n",
    "    # Criar histograma\n",
    "    plt.figure(figsize=(12, 5))\n",
    "    \n",
    "    # Subplot 1: Histograma das arestas\n",
    "    plt.subplot(1, 2, 1)\n",
    "    plt.hist(edge_lengths, bins=50, alpha=0.7, color='skyblue', edgecolor='black')\n",
    "    plt.axvline(np.mean(edge_lengths), color='red', linestyle='--', label=f'Média: {np.mean(edge_lengths):.2f}')\n",
    "    plt.axvline(np.median(edge_lengths), color='green', linestyle='--', label=f'Mediana: {np.median(edge_lengths):.2f}')\n",
    "    plt.title(f\"Distribuição do Comprimento das Arestas\\n{TARGET_STL_FILENAME}\")\n",
    "    plt.xlabel(\"Comprimento da Aresta (mm)\")\n",
    "    plt.ylabel(\"Frequência\")\n",
    "    plt.legend()\n",
    "    plt.grid(True, alpha=0.3)\n",
    "    \n",
    "    # Subplot 2: Distribuição dos vértices por eixo\n",
    "    plt.subplot(1, 2, 2)\n",
    "    vertices = mesh_simplified.vertices\n",
    "    plt.scatter(vertices[:, 0], vertices[:, 1], s=1, alpha=0.6, label='Projeção XY')\n",
    "    plt.xlabel(\"X (mm)\")\n",
    "    plt.ylabel(\"Y (mm)\")\n",
    "    plt.title(\"Distribuição dos Vértices (Vista Superior)\")\n",
    "    plt.axis('equal')\n",
    "    plt.grid(True, alpha=0.3)\n",
    "    \n",
    "    plt.tight_layout()\n",
    "    \n",
    "    # Salvar histograma\n",
    "    hist_path = os.path.join(RESULTS_DIR, f\"{os.path.splitext(TARGET_STL_FILENAME)[0]}_analysis.png\")\n",
    "    plt.savefig(hist_path, dpi=300, bbox_inches='tight')\n",
    "    print(f\"\\n✅ Análise geométrica salva em: {hist_path}\")\n",
    "    \n",
    "    # Mostrar o gráfico\n",
    "    plt.show()\n",
    "    \n",
    "    # Análise da qualidade da malha\n",
    "    print(f\"\\nQualidade da Malha:\")\n",
    "    print(f\"Watertight: {mesh_simplified.is_watertight}\")\n",
    "    print(f\"Orientação consistente: {mesh_simplified.is_winding_consistent}\")\n",
    "    print(f\"Tem self-intersections: {mesh_simplified.is_self_intersecting}\")\n",
    "    \n",
    "    # Bounding box\n",
    "    bounds = mesh_simplified.bounds\n",
    "    print(f\"\\nBounding Box:\")\n",
    "    print(f\"X: [{bounds[0,0]:.2f}, {bounds[1,0]:.2f}] mm (largura: {bounds[1,0]-bounds[0,0]:.2f} mm)\")\n",
    "    print(f\"Y: [{bounds[0,1]:.2f}, {bounds[1,1]:.2f}] mm (profundidade: {bounds[1,1]-bounds[0,1]:.2f} mm)\")\n",
    "    print(f\"Z: [{bounds[0,2]:.2f}, {bounds[1,2]:.2f}] mm (altura: {bounds[1,2]-bounds[0,2]:.2f} mm)\")\n",
    "    \n",
    "else:\n",
    "    print(\"❌ Malha simplificada não disponível para análise adicional\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Conclusão da Exploração\n",
    "\n",
    "Este notebook demonstrou o carregamento, visualização e simplificação de um modelo de crânio STL usando as ferramentas desenvolvidas no projeto.\n",
    "\n",
    "### Principais Resultados:\n",
    "\n",
    "1. **Carregamento eficiente**: O sistema de cache acelera significativamente operações repetidas\n",
    "2. **Simplificação controlada**: A decimação quadrática preserva as características importantes da malha\n",
    "3. **Visualização clara**: As projeções 2D permitem análise visual em qualquer ambiente\n",
    "4. **Análise quantitativa**: Estatísticas detalhadas sobre a geometria da malha\n",
    "\n",
    "### Próximos Passos:\n",
    "\n",
    "- A malha simplificada está pronta para detecção de landmarks\n",
    "- O sistema de cache otimizará processamentos futuros\n",
    "- As visualizações ajudam a validar a qualidade do pré-processamento\n",
    "\n",
    "**A etapa de pré-processamento foi validada como funcional e eficiente.**"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}